In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from shutil import copyfile

In [2]:
def err_ratio(data,err):
    data[2:-2,2:-2]=0
    err[2:-2,2:-2]=0
    edge_fluxmap = data[data!=0]
    edge_errmap = err[err!=0]
    #data[data == 0.01] = 0
    #err[err < 10**(-14)] =10**(10)  
    #return data,err
    err_fluxmap = np.std(edge_fluxmap)
    err_errmap = np.mean(edge_errmap)
    fact = err_fluxmap/err_errmap
    return fact

def emp_fact(err_ratio_Hb1,err_ratio_Hb2,err_ratio_OIII_br,broad2=False):
    if broad2:
        emp_Hb2 = err_ratio_Hb2
    else:
        emp_Hb2 = 0
    emp_Hb1 = err_ratio_Hb1
    emp_wing = err_ratio_OIII_br
    return emp_Hb1,emp_Hb2,emp_wing

In [3]:
def emp_table(obj,emp_Hb1,emp_Hb2,emp_wing,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    popt = [emp_Hb1,emp_Hb2,emp_wing]
    column_names={'emp_fact_Hb1':0,'emp_fact_Hb2':1,'emp_fact_wing':2}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[popt[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_9_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj),overwrite=True)

def emp_fact_table(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_9_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_9_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj),overwrite=True)

In [4]:
def algorithm_script(obj,z,broad2,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))

    (Hb1_blr_br_data,Hb2_blr_br_data) = loadblr('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj))
    (Hb1_blr_br_err,Hb2_blr_br_err) = loadblr('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_err = loadwing('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_data = loadwing('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj)) # It opens the hdu[3] where the OIII_br data 
  
    err_ratio_OIII_br = err_ratio(OIII_br_data,OIII_br_err)
    err_ratio_Hb1 = err_ratio(Hb1_blr_br_data,Hb1_blr_br_err)
    err_ratio_Hb2 = err_ratio(Hb2_blr_br_data,Hb2_blr_br_err)
    (emp_Hb1,emp_Hb2,emp_wing) = emp_fact(err_ratio_Hb1,err_ratio_Hb2,err_ratio_OIII_br,broad2)
    print (emp_Hb1,emp_Hb2,emp_wing)
    emp_table(obj,emp_Hb1,emp_Hb2,emp_wing)
   # emp_fact_table(obj)

In [5]:
z = {"HE0021-1810":0.05352}

broad2= {"HE0021-1810":False}

objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0021-1810
1.2778674 0 1.4592088


In [7]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}


objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0021-1810
1.3658861 0 1.6496661
HE0021-1819
1.5791438 0 1.3521601
HE0040-1105
1.65103 0 1.6617692
HE0108-4743
1.342079 2.024678 2.135341
HE0114-0015
1.596306 0 1.7333821
HE0119-0118
1.4236484 1.525203 1.3894937
HE0212-0059
1.4469695 1.0959408 1.1555918
HE0224-2834
1.7291038 0 1.8229746
HE0227-0913
2.183495 1.3780653 1.4944391
HE0232-0900
1.86127 0 5.1029305
HE0253-1641
1.5903094 1.6383115 1.1833996
HE0345+0056
1.6202188 1.3942055 1.2192378
HE0351+0240
1.2850196 1.5067376 2.4876807
HE0412-0803
1.4144753 0 3.7227354
HE0429-0247
1.2741908 1.2756499 1.5495409
HE0433-1028
1.2863921 1.1571751 1.3452921
HE0853+0102
1.4686188 1.4781276 1.4976017
HE0934+0119
2.282569 1.8362265 1.6719517
HE1011-0403
1.8384334 1.6742551 1.3460308
HE1017-0305
1.5528755 0 1.2176002
HE1029-1831
1.8568946 1.6701672 1.9236847
HE1107-0813
1.3343713 1.5722094 1.3224044
HE1108-2813
1.8106674 0 1.4972596
HE1126-0407
1.703259 1.630469 1.2802953
HE1237-0504
2.2641804 0 2.3462694
HE1248-1356
1.7083539 0 1.5585883
HE1330-10

For this galaxy when we did not have the proper error propagation estimating code 
and continuum missing from the broad H-beta the reduced chi-squared was 23. Introducing 
the continuum brought it down to 14. Then we did the error estimation with monte carlo 
properly which further brought it down to 7. Now we see that the error for the both 
BLR H-beta and broad [OIII] was underestimated. 

What we did here is to check the error from the 2D maps and the 2D error maps to see whether
they make any sense. That is why we choose the region of the edge where we took 2 pixels from 
the edges along the map and calculate the standard deviation. The reason behind this is BLR
H-beta is point like and hence along the edge it should be almost equal to 0 but from the 2D
maps it is not cleraly 0 but a fluctuation between positive and sometimes negative values.
Taking the standard deviation would give us the error.

Next we take the 2D error maps and choose the corresponding pixels. Since this is an error map
already, thi stime we calculate the mean. Taking the reatios of this two error we get a ratio of
5 which means that the true error was underestimated to a factor of 5. This factor is constant 
throughout the edge.

We also do it for the broad [OIII] since most of the [OIII] emission comes from a very compact
region around the center. Hence, if there is any excess flux that should be rather low and will
not interfere with our measurements. Repeating the previous steps we get a ratio of 2.5.

We cannot do it for the narrow [OIII] since it is extended in nature and hence the error computation
from the 2D maps will not return its true eeror.